In [88]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

from collections import Counter
from scipy import stats
from math import log2, sqrt
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn import svm
%load_ext autoreload

%autoreload 2

%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.layers import LeakyReLU
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from google.colab import drive
drive.mount('/content/gdrive')
import keras as ks
from PIL import Image



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [93]:
batch_size = 32
init_lr= 0.006
directory = '/content/gdrive/MyDrive/HackGT 9 Project/allpics'

model = None
def preprocess(img_dir, img_file):
  img = tf.keras.utils.img_to_array(tf.keras.utils.load_img(img_dir + '/' + img_file))
  img = img.astype(np.float32) - np.mean(img)
  img /= np.std(img)
  return img  

def load_data(img_dir, num_classes):
  goodnum = 0
  imgs = []
  labels = []
  for img_file in listdir(img_dir):
    img = preprocess(img_dir, img_file)
    shape = np.shape(img)
    if shape[1] < 112:
      img = np.concatenate((img, np.zeros((112, 112 - shape[1], 3))), axis = 1)
    if shape[0] < 112:
      img = np.concatenate((img, np.zeros((112 - shape[0], 112, 3))), axis = 0)
    goodnum = goodnum + 1
    imgs.append(img)
    label = int(img_file.split('_')[0])
    labels.append(np.eye(num_classes)[label])
  return imgs, labels

model = Sequential()

num_classes = 7
imgs, labels = load_data(directory, num_classes)
model.add(Conv2D(8, kernel_size=3, padding='same', input_shape = (112, 112, 3)))
model.add(LeakyReLU())
model.add(Conv2D(32, kernel_size=3, padding= 'same')) 
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(32, kernel_size=3, padding= 'same'))
model.add(LeakyReLU())
model.add(Conv2D(64, kernel_size=3, padding= 'same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dropout(0.3))
model.add(Dense(7))
model.add(Activation('softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(len(imgs))
model.fit(np.asarray(imgs), np.asarray(labels), epochs = 10, verbose = 0)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
open('object_recognition.tflite', 'wb').write(converter.convert())




        

486


51515152

In [94]:
ip = tf.lite.Interpreter(model_path="object_recognition.tflite")
ip.allocate_tensors()
input_index = ip.get_input_details()[0]['index']
output_index = ip.get_output_details()[0]['index']

def testnetwork(img_dir):
  num_correct = 0
  for img_file in listdir(img_dir):
    img = preprocess(img_dir, img_file)
    shape = np.shape(img)
    if shape[1] < 112:
      img = np.concatenate((img, np.zeros((112, 112 - shape[1], 3))), axis = 1)
    if shape[0] < 112:
      img = np.concatenate((img, np.zeros((112 - shape[0], 112, 3))), axis = 0)
    ip.set_tensor(input_index, ([img.astype(np.float32)]))
    ip.invoke()
    preds = ip.get_tensor(output_index)
    label = int(img_file.split('_')[0])
    if np.argmax(preds) == label:
      num_correct += 1
    num_imgs = len(listdir(img_dir))
  return('{} correct out of {}'.format(num_correct, num_imgs))

print('Verifying the network on the dataset used to train it:')
print(testnetwork('/content/gdrive/MyDrive/HackGT 9 Project/allpics'))
print('Testing the network on a test dataset:')
print(testnetwork('/content/gdrive/MyDrive/HackGT 9 Project/testpics'))

Verifying the network on the dataset used to train it:
486 correct out of 486
Testing the network on a test dataset:
11 correct out of 18
